In [3]:
import selenium
from selenium import webdriver as wd
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import repeat

In [8]:
# 크롬드라이버 열기
driver = wd.Chrome('/Users/sophiekim/Downloads/chromedriver-mac-arm64/chromedriver') # 크롬드라이버 경로
driver.maximize_window() # 크롬창 크기 최대

# 드라이버가 해당 url 접속
url = 'https://www.melon.com/chart/index.htm' # 멜론차트 페이지
driver.get(url)

# 차트파인더 클릭
driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span').click()

# 연대선택, 연도선택, 월선택, 장르선택

# 주간차트 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[1]/a').click()
time.sleep(2)

# 연대선택 1980년 클릭
# 90년대 //*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li"[4]"/span/label >> "" 부분 숫자만 바뀜
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[5]/span/label').click()
time.sleep(2)

# 연도선택 1984년 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[6]/span/label').click()
time.sleep(2)

# 월선택 3월 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[1]/span/label').click()
time.sleep(2)

# 주간선택
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li[1]/span/label').click()
time.sleep(1)

# 장르선택 종합 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li/span/label').click()
time.sleep(2)

# 검색버튼 클릭
driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span').click()

In [9]:
html = driver.page_source # 드라이버 현재 페이지의 html 정보 가져오기 
                            # cf) requests.get(url)
soup = BeautifulSoup(html, 'lxml')

노래 제목

In [48]:
titles = soup.find_all('div', attrs={'class':'ellipsis rank01'})
song_name = []
for title in titles:
    try:
        name = title.find('a').text
    except:
        name = title.find('span', attrs={'class':'fc_lgray'}).text
    song_name.append(name)
    
print(song_name)

['못다핀 꽃 한 송이', '슬픔의 심로', '친구여', '고니', '하얀 목련', '그대가 미워요', '그대는 나의 인생', '아파트', '은지', '얼룩진 상처', '고독', '사랑은 연필로 쓰세요', '빗물', '불씨', '우는 아인 바보야', '어머니와 고등어', '정녕 그대를', '고독한 여자의 미소는 슬퍼', '유리벽', '작은 숙녀', '도요새의 비밀', '그래 걷자', '이별 뒤의 이야기', '지울수 없는 얼굴']


가수

In [62]:
singers = soup.find_all('span', attrs={'class':'checkEllipsis'})
singer_name = []
for singer in singers:
    name = singer.find('a', attrs={'class':'fc_mgray'}).text
    singer_name.append(name)

print(singer_name)

['김수철', '김학래', '조용필', '이태원', '양희은', '전영록', '한울타리', '윤수일', '배따라기', '백영규', '최백호', '전영록', '송골매', '신형원', '하남석', '김창완', '김수철', '김정호', '신형원', '혜은이', '정광태', '김창완', '이용', '채은옥']


앨범명

In [67]:
albums = soup.find_all('div', attrs={'class':'ellipsis rank03'})
album_list = []
for album in albums:
    a = album.find('a', attrs={'class':'fc_mgray'}).text
    album_list.append(a)

print(album_list)

['작은 거인 김수철 (못다핀 꽃 한 송이)', '김학래 전집', '조용필 5집', '타조, 솔개, 여인아', 'Memories', '사랑은 연필로 쓰세요', '눈물의 승차권/그대는 나의인생', '골든 힛트곡 모음집 (사랑만은 않겠어요)', '배따라기', '백영규 9집', '83 최백호', '사랑은 연필로 쓰세요', '송골매 3집', '신형원의 모든 것 1', '우는 아이 바보야', '김창완 - 기타가 있는 수필', '작은 거인 김수철 (못다핀 꽃 한 송이)', '김정호 골든', '신형원 베스트 1982-2000', '작은 숙녀', '정광태 2집', '김창완 - 기타가 있는 수필', '1981~2003 History (New & Best Live Version)', '지울수 없는 얼굴/사랑할줄 모르고']


데이터프레임 만들기

In [98]:
df = pd.DataFrame({
    'Rank' : list(range(1, len(song_name)+1)),
    'Title' : song_name,
    'Singer' : singer_name,
    'Album' : album_list

})

In [99]:
df

,Rank,Title,Singer,Album
0,1,못다핀 꽃 한 송이,김수철,작은 거인 김수철 (못다핀 꽃 한 송이)
1,2,슬픔의 심로,김학래,김학래 전집
2,3,친구여,조용필,조용필 5집
3,4,고니,이태원,"타조, 솔개, 여인아"
4,5,하얀 목련,양희은,Memories
5,6,그대가 미워요,전영록,사랑은 연필로 쓰세요
6,7,그대는 나의 인생,한울타리,눈물의 승차권/그대는 나의인생
7,8,아파트,윤수일,골든 힛트곡 모음집 (사랑만은 않겠어요)
8,9,은지,배따라기,배따라기
9,10,얼룩진 상처,백영규,백영규 9집
